필요한 모듈 다운

In [ ]:
#!pip3 install openpyxl # xlsx파일을 읽기위한 모듈
#!pip3 install pandas
import pandas as pd
import os
import re

/Users/joonseok/Documents/train_Dataset/ 라는 경로에다 사용할 파일들을 두기


모기 데이터 파일은 mos_d.xlsx 이라는 엑셀 파일로
날씨 데이터 파일들은 csv파일으로 wea_d_dir 라는 폴더 안에 넣어둔다
수질 데이터 파일들은 엑셀파일로 wat_d_dir 라는 폴더 안에 넣어둔다


raw 데이터들을 저장할건데
모기 데이터셋을 r_mos_2017~2022, 
날씨 데이터셋을 r_wea_2017~2022, 
수질 데이터셋을 r_wat_2017~2022 
변수들에다가 저장

In [ ]:
# 모기 데이터 시트별로 나눠져 있는 데이터를 r_mos_2018 이런 식으로 분리
for i in range(6):
    globals()['r_mos_' + str(2022 - i)] = pd.read_excel("/Users/joonseok/Documents/train_Dataset/mos_d.xlsx", sheet_name=i)
# 출력
for i in range(6):
    print('\nvar name : r_mos_' + str(2017 + i)+'\t==========')
    mo = globals()['r_mos_' + str(2017 + i)]
    print(mo)

In [ ]:
# 날씨 데이터 시트별로 나눠져 있는 데이터를 r_wea_2018 이런 식으로 분리
for i in os.listdir(r'/Users/joonseok/Documents/Dataset/wea_d_dir/') :
    if("weather" in i):
        globals()['r_wea_'+re.findall(r'\d{4}',i)[0]] = pd.read_csv("/Users/joonseok/Documents/train_Dataset/wea_d_dir/"+i, encoding='cp949')
# 출력
for i in range(6) :
    print('\nvar name : r_wea_'+str(2017+i)+'\t==========')
    we = globals()['r_wea_'+str(2017+i)]
    print(we)

In [ ]:
# 수질 데이터 시트별로 나눠져 있는 데이터를 r_wat_2018 이런 식으로 분리
for i in os.listdir(r'/Users/joonseok/Documents/Dataset/wat_d_dir/') :
    if("water" in i):
        globals()['r_wat_'+re.findall(r'\d{4}',i)[0]] = pd.read_excel("/Users/joonseok/Documents/train_Dataset/wat_d_dir/"+i)
# 출력
for i in range(6) :
    print('\nvar name : r_wat_'+str(2017+i)+'\t==========')
    wa = globals()['r_wat_'+str(2017+i)]
    print(wa)

모기 데이터 셋은 날짜와 모기데이터만 추출

In [ ]:
# 사용할 열 이름
u_col = ['날짜','모기']
# 필요한 데이터만 추출
for i in range(6):
    mo = globals()['r_mos_' + str(2017 + i)]
    mos = mo[u_col]
    mos['날짜'] = pd.to_datetime(mos['날짜'], format='%Y%m%d')
    globals()['mos_' + str(2017 + i)] = mos

In [ ]:
# 출력
for i in range(6):
    print('\nvar name : mos_' + str(2017 + i)+'\t==========')
    mo = globals()['mos_' + str(2017 + i)]
    print(mo)

날씨 데이터셋은 슬라이싱으로 필요한 행(5월부터 10월까지의 데이터), 열(날짜, 평균기온, 일강수량, 평균 풍속, 평균 상대 습도,평균 증기압, 합계 일사시간, 합계 일사량, 평균 지면온도) 추출

In [ ]:
# 사용할 열 이름
u_col = ['일시', '평균기온(°C)', '일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','평균 증기압(hPa)','합계 일조시간(hr)','합계 일사량(MJ/m2)','평균 지면온도(°C)']
# 필요한 데이터만 추출
for i in range(6):
    we = globals()['r_wea_' + str(2017 + i)]
    wea = we[u_col]
    wea['일시'] = pd.to_datetime(wea['일시'])
    #통합할때 편하게 일시를 날짜로 이름 변경
    wea = wea.rename(columns={'일시':'날짜'})
    # 비 안오는날인 강수량 Na값을 0으로 채움
    wea['일강수량(mm)'] = wea['일강수량(mm)'].fillna(0)
    globals()['wea_' +  str(2017 + i)] = wea

In [ ]:
# 출력
for i in range(6):
    print('\nvar name : wea_' + str(2017 + i)+'\t==========')
    we = globals()['wea_' + str(2017 + i)]
    print(we)

수질 데이터셋은 슬라이싱으로 필요한 열만 추출 한 후 필요한 행만 추출 하고 날짜를 기준으로 데이터들의 평균을 구해 모든 강의 일별 평균 수질데이터를 추출

In [ ]:
#사용할 열 이름
u_col = ['날짜', '수온', 'pH', '용존산소(㎎/L)']
for i in range(6) :
    wa = globals()['r_wat_'+str(2017+i)]
    wat = wa[u_col]
    wat['날짜'] = pd.to_datetime(wat['날짜'], format='%Y%m%d')
    # 분리할 날짜들
    s_date = pd.to_datetime(str(2017+i)+'-05-01')  # 시작 날짜
    e_date = pd.to_datetime(str(2017+i)+'-10-31')   # 종료 날짜
    row_data = wat[(wat['날짜'] >= s_date) & (wat['날짜'] <= e_date)]
    # 데이터 merge시키면서 inner join 시키면 되서 굳이 행을 안지워도 되지만 수질데이터가 너무 많으니까 행을 지워서 데이터를 줄여줌
    # 날짜에 따라 데이터 평균 구하기
    col_data = row_data.groupby('날짜').mean()                                             #그룹을 시키면서 컬럼에서 날짜가 빠짐
    globals()['wat_' + str(2017 + i)] = col_data

In [ ]:
# 출력
for i in range(6):
    print('\nvar name : wat_' + str(2017 + i)+'\t==========')
    wa = globals()['wat_' + str(2017 + i)]
    print(wa)

각 데이터 전처리는 완료했으니 이제 데이터셋을 data_2017~2022로 merge 시키고 하나의 dataset으로 만든다

In [ ]:
for i in range(6):
    mo = globals()['mos_' + str(2017 + i)]
    we = globals()['wea_' + str(2017 + i)]
    wa = globals()['wat_' + str(2017 + i)]
    mo_we = pd.merge(mo, we, on='날짜', how='inner')
    da = pd.merge(mo_we, wa, on='날짜', how='inner')
    globals()['data_' + str(2017 + i)] = da 

dataset = pd.concat([data_2017, data_2018], axis=0)
dataset = pd.concat([dataset, data_2019], axis=0)
dataset = pd.concat([dataset, data_2020], axis=0)
dataset = pd.concat([dataset, data_2021], axis=0)
dataset = pd.concat([dataset, data_2022], axis=0)

통합 데이터셋을 출력해서 확인한다

In [ ]:
dataset

데이터셋에서 NA값이 얼마나 있는지 확인한다

In [ ]:
print(dataset.isnull().sum())

모기의 Na값은 장비점검으로 인해 2주간 측정이 안된 것이고 수질데이터는 장비이상으로 5일간 측정이 안된것으로 이 기간들은 데이터에서 제외

In [ ]:
dataset = dataset.dropna(axis=0)
dataset

outlier값을 찾아서 제거해준다

In [ ]:
q3 = dataset.quantile(0.75)
q1 = dataset.quantile(0.25)
iqr = q3 - q1

In [ ]:
# '합계 일사량' 열에 대하여 이상치 여부를 판별해주는 함수
def is_solar_outlier(df):
    solar = df['합계 일사량(MJ/m2)']
    if solar > q3['합계 일사량(MJ/m2)'] + 1.8 * iqr['합계 일사량(MJ/m2)'] or solar < q1['합계 일사량(MJ/m2)'] - 1.8 * iqr['합계 일사량(MJ/m2)']:
        return True
    else:
        return False

# apply 함수를 통하여 각 값의 이상치 여부를 찾고 새로운 열에 결과 저장
dataset['is_solar_out'] = dataset.apply(is_solar_outlier, axis = 1)

# 'ph' 열에 대하여 이상치 여부를 판별해주는 함수
def is_ph_outlier(df):
    ph = df['pH']
    if ph > q3['pH'] + 1.8 * iqr['pH'] or ph < q1['pH'] - 1.8 * iqr['pH']:
        return True
    else:
        return False

dataset['is_ph_out'] = dataset.apply(is_ph_outlier, axis = 1)

# 이상치인 행은 제거하여 필터링
dataset = dataset.loc[dataset['is_solar_out'] == False]

# 이상치여부를 나타내는 열 제거
del dataset['is_solar_out']

dataset = dataset.loc[dataset['is_ph_out'] == False]
del dataset['is_ph_out']

이상치 제거된 데이터셋 출력

In [ ]:
dataset

In [ ]:
dataset.rename(columns={'날짜':'Date','모기':'Mosquito','평균기온(°C)':'Averagetemperature','일강수량(mm)':'Dailyprecipitation',
                        '평균 풍속(m/s)':'Averagewindspeed','평균 상대습도(%)':'Averagerelativehumidity','평균 증기압(hPa)':'Averagevaporpressure',
                        '합계 일조시간(hr)':'Totalsunshinehours','합계 일사량(MJ/m2)':'Totalsolarradiation','평균 지면온도(°C)':'Averagegroundtemperature',
                        '수온':'Watertemperature','pH':'Ph','용존산소(㎎/L)':'Dissolvedoxygen'},inplace=True)

In [ ]:
dataset

sklearn에서 LabelEncoder의 라벨링 기능을 이용해 Date에서 연도랑 시간을 제외하고 날짜만 남겨서 라벨링을 해줘서 Numeric한 데이터로서 사용할 수 있게 바꿔준다

In [ ]:
#!pip3 install scikit-learn
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
dataset['Date_str'] = pd.to_datetime(dataset['Date']).dt.month.astype(str).str.zfill(2) + pd.to_datetime(dataset['Date']).dt.day.astype(str).str.zfill(2)
dataset['Date_Labeled'] = le.fit_transform(dataset['Date_str'])
# 확인용 컬럼이였던 Date_str 컬럼 삭제
dataset = dataset.drop(['Date_str'], axis=1)

데이터셋 csv파일로 출력

In [ ]:
dataset.to_csv('/Users/joonseok/Documents/Dataset/training_dataset.csv', index=False, encoding='CP949')